In [1]:
%cd ..

c:\Users\HuyLQ15_CTV\Desktop\model_monitoring


In [295]:
from sklearn.datasets import load_iris
data = load_iris(as_frame=True)['data']
data['target'] = load_iris(as_frame=True)['target']

In [296]:
from sklearn.linear_model import LogisticRegression

X = data.drop('target', axis=1).iloc[:125]
y = data['target'].iloc[:125]
model = LogisticRegression()
model.fit(X, y)
prediction1 = model.predict(data.drop('target', axis=1))
prediction2 = model.predict_proba(data.drop('target', axis=1))
data['prediction'] = prediction1
data[[0,1,2]] = prediction2

In [297]:
import pandas as pd
data2 = pd.DataFrame(prediction2, columns=model.classes_)
data2['target'] = data['target']

In [298]:
reference = data.iloc[:125]
current = data.iloc[125:]

In [299]:
labels=[0,1,2]

# Kế hoạch
Có 2 option để truyền vào column mapping
- Kết quả dạng đã chuyển thành predict label
- Kết quả dạng xác suất


## Transformed labels

### Utils

In [257]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

classification_report(data['target'], data['prediction'], output_dict=True).pop('accuracy')

0.9533333333333334

In [258]:
from source.calculator.classification_performance import calculate_confusion_by_classes 
from sklearn.metrics import confusion_matrix
def calculate_confusion_matrix_by_classes(y_true, y_pred):
    labels = list(set(data['target']) | set(data['prediction']))
    confusion_matrix_inf = confusion_matrix(y_true, y_pred, labels=labels)
    return calculate_confusion_by_classes(confusion_matrix_inf, class_names=labels)

calculate_confusion_matrix_by_classes(current['target'], current['prediction'])


{0: {'tp': 0.0, 'tn': 25.0, 'fp': 0.0, 'fn': 0.0},
 1: {'tp': 0.0, 'tn': 21.0, 'fp': 4.0, 'fn': 0.0},
 2: {'tp': 21.0, 'tn': 0.0, 'fp': 0.0, 'fn': 4.0}}

In [259]:
from sklearn.metrics import matthews_corrcoef
matthews_corrcoef(data['target'], data['prediction'])

np.float64(0.9330529677131488)

### Metrics

In [ ]:
from evidently.base_metric import MetricResult, InputData
class MultiClassificationResult(MetricResult):
    class Config:
        type_alias = "evidently:metric_result:MultiClassificationResult"
    accuracy: float
    confusion_matrix: dict
    classification_info: dict
    logloss: Optional[float]
    roc_auc: Optional[dict]
    mattheus_corr_coefficient: float

def calculate_multi_classification(
    prediction: pd.Series, 
    target: pd.Series,
    prediction_proba: pd.DataFrame = None
    ) -> MultiClassificationResult:
    # Initialize logloss and roc_auc as None
    logloss = None
    roc_auc = None
    accuracy = accuracy_score(target, prediction)
    classification_info = classification_report(target, prediction, output_dict=True)
    classification_info.pop('accuracy')
    confusion_matrix = calculate_confusion_matrix_by_classes(target, prediction)
    mattheus_corr_coefficient = matthews_corrcoef(target, prediction)
    if prediction_proba is not None:
        roc_auc = {}
        roc_auc['ovr'] = float(roc_auc_score(y_true=prediction, y_score=prediction_proba, multi_class='ovr'))
        roc_auc['ovo'] = float(roc_auc_score(y_true=prediction, y_score=prediction_proba, multi_class='ovo'))
        logloss = log_loss(y_true=prediction, y_pred=prediction_proba)
    return MultiClassificationResult(
        accuracy = accuracy,
        confusion_matrix = confusion_matrix,
        classification_info = classification_info,
        logloss = logloss,
        roc_auc = roc_auc,
        mattheus_corr_coefficient = float(mattheus_corr_coefficient)
    )

In [314]:
from typing import Optional
from sklearn.metrics import accuracy_score
from sklearn.metrics import matthews_corrcoef
from evidently.base_metric import Metric
from evidently.base_metric import MetricResult, InputData
from evidently.utils.data_operations import process_columns
from evidently.calculations.classification_performance import get_prediction_data



class MultiClassificationMetric(Metric[MultiClassificationResult]):
    class Config:
        type_alias = "evidently:metric:MultiClassificationMetric"
    _threshold: float
    
    def __init__(self, threshold=0.5):
        self._threshold = threshold
        super().__init__()

    def calculate(self, data: InputData) -> MultiClassificationResult:
        column_mapping = data.column_mapping
        reference = data.reference_data
        current = data.current_data

        current.columns = current.columns.map(str)
        dataset_column = process_columns(current, column_mapping)
        prediction_data = get_prediction_data(current, data_columns=dataset_column, pos_label=None, threshold = self._threshold)
        prediction_data.predictions = prediction_data.predictions.map(float)
        return calculate_multi_classification(
            prediction=prediction_data.predictions,
            target=current[column_mapping.target],
            prediction_proba=prediction_data.prediction_probas
        )


c:\Users\HuyLQ15_CTV\AppData\Local\miniconda3\envs\model-monitoring\lib\site-packages\evidently\pydantic_utils.py:161: UserWarning: Duplicate key (<class 'evidently.base_metric.MetricResult'>, 'evidently:metric_result:MultiClassificationResult') in alias map
  warnings.warn(f"Duplicate key {key} in alias map")
c:\Users\HuyLQ15_CTV\AppData\Local\miniconda3\envs\model-monitoring\lib\site-packages\evidently\pydantic_utils.py:161: UserWarning: Duplicate key (<class 'evidently.base_metric.Metric'>, 'evidently:metric:MultiClassificationMetric') in alias map
  warnings.warn(f"Duplicate key {key} in alias map")


In [315]:
from evidently.renderers.base_renderer import MetricRenderer, default_renderer

@default_renderer(wrap_type=MultiClassificationMetric)
class MultiClassificationRenders(MetricRenderer):
    def render_json(self, obj: MultiClassificationMetric, include_render: bool = False,
        include: "IncludeOptions" = None, exclude: "IncludeOptions" = None,) -> dict:
        result = obj.get_result().get_dict(include_render, include, exclude)
        return result

In [316]:
current.columns = current.columns.map(str)
column_mapping = ColumnMapping()
column_mapping.target = 'target'
column_mapping.prediction = ['0','1','2']
dataset_column = process_columns(current, column_mapping)
prediction_data = get_prediction_data(current, data_columns=dataset_column, pos_label=None, threshold =0.5)
prediction_data.prediction_probas

,0,1,2
125,0.000012,0.150204,0.849784
126,0.001227,0.603214,0.395559
127,0.001229,0.544907,0.453864
128,0.000023,0.096845,0.903132
129,0.000032,0.312453,0.687515
130,0.000003,0.093674,0.906323
131,0.000002,0.073749,0.926248
132,0.000018,0.076238,0.923744
133,0.000617,0.627657,0.371726
134,0.000098,0.334062,0.665839


In [317]:
from evidently.report import Report

report = Report(metrics=[
    MultiClassificationMetric()
])

report.run(current_data=current, reference_data=None, column_mapping=column_mapping)
report.json()

c:\Users\HuyLQ15_CTV\AppData\Local\miniconda3\envs\model-monitoring\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\HuyLQ15_CTV\AppData\Local\miniconda3\envs\model-monitoring\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\HuyLQ15_CTV\AppData\Local\miniconda3\envs\model-monitoring\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_

ValueError: Number of classes in y_true not equal to the number of columns in 'y_score'

In [191]:
current.columns = current.columns.map(str)

In [192]:
column_mapping = ColumnMapping()
column_mapping.prediction = ['0','1','2']
column_mapping.target = 'target'

In [193]:
dataset_column = process_columns(current, column_mapping)
prediction_data = get_prediction_data(current, data_columns=dataset_column, pos_label=None, threshold = 0.5)
prediction_data

PredictionData(type='evidently:metric_result:PredictionData', predictions=125    2
126    1
127    1
128    2
129    2
130    2
131    2
132    2
133    1
134    2
135    2
136    2
137    2
138    1
139    2
140    2
141    2
142    2
143    2
144    2
145    2
146    2
147    2
148    2
149    2
dtype: object, labels=[0, 1, 2], prediction_probas=                0         1         2
125  4.514727e-06  0.092114  0.907882
126  1.348251e-03  0.757529  0.241122
127  1.381911e-03  0.729877  0.268741
128  7.645899e-06  0.055618  0.944374
129  1.092012e-05  0.159804  0.840186
130  1.297124e-06  0.061696  0.938303
131  1.273513e-06  0.067459  0.932540
132  6.158512e-06  0.046733  0.953261
133  7.263059e-04  0.771156  0.228118
134  2.640124e-05  0.136890  0.863083
135  6.908874e-07  0.040611  0.959388
136  1.003903e-05  0.041155  0.958835
137  3.314951e-05  0.124336  0.875631
138  1.939524e-03  0.751611  0.246449
139  2.323922e-05  0.119336  0.880641
140  5.875843e-06  0.045684  0.954310
141 

## Probability label

In [87]:
prediction_proba = data[['1', '2', '0']]
prediction = data['prediction']

In [93]:
from sklearn.metrics import roc_auc_score, log_loss
roc_auc_score(y_true=prediction, y_score=prediction_proba, multi_class='ovr')
log_loss(y_true=prediction, y_pred=prediction_proba)

5.73506311657307

# Test

In [164]:
from evidently.pipeline.column_mapping import ColumnMapping

column_mapping = ColumnMapping()
column_mapping.prediction = ['0','1','2']
column_mapping.target = 'target'


In [166]:
column_mapping

ColumnMapping(target='target', prediction=['0', '1', '2'], datetime='datetime', id=None, numerical_features=None, categorical_features=None, datetime_features=None, target_names=None, task=None, pos_label=1, text_features=None, embeddings=None, user_id='user_id', item_id='item_id', recommendations_type=<RecomType.SCORE: 'score'>)

In [167]:
from evidently.utils.data_operations import process_columns
from evidently.calculations.classification_performance import get_prediction_data

dataset_column = process_columns(data, column_mapping)
prediction_data = get_prediction_data(data, data_columns=dataset_column, pos_label=None, threshold = 0.5)

In [168]:
prediction_data

PredictionData(type='evidently:metric_result:PredictionData', predictions=0      0
1      0
2      0
3      0
4      0
      ..
145    2
146    2
147    2
148    2
149    2
Length: 150, dtype: object, labels=[0, 1, 2], prediction_probas=            0         1             2
0    0.984358  0.015642  9.935675e-08
1    0.977970  0.022029  2.041954e-07
2    0.987768  0.012232  9.750679e-08
3    0.980500  0.019499  2.500493e-07
4    0.986975  0.013025  8.397178e-08
..        ...       ...           ...
145  0.000030  0.101245  8.987248e-01
146  0.000078  0.168555  8.313668e-01
147  0.000059  0.146487  8.534546e-01
148  0.000028  0.064857  9.351151e-01
149  0.000151  0.167599  8.322504e-01

[150 rows x 3 columns])

In [ ]:
from evidently.metric_results import DatasetColumns
from evidently.pipeline.column_mapping import ColumnMapping